# Assinatura Digital

#### Alunos: Gabriel Schneider e Luciane Tedesco 

#### Questão 1

- Crie dois pares de chaves (par de chaves A e par de chaves B) e armazene-os em disco.

In [24]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend

private_key_A = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096,
    backend=default_backend()
)

private_key_B = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096,
    backend=default_backend()
)

public_key_A = private_key_A.public_key()
public_key_B= private_key_B.public_key()

In [33]:
from cryptography.hazmat.primitives import serialization
print(public_key_A)

pem = private_key_A.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.BestAvailableEncryption(b'mypassword')
)

f = open('public.pem', 'wb')
f.write(pem)
f.close()

#https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/

In [28]:
# salvando arquivos
f = open('public.pem', 'wb')
f.write(public_key_A)
f.close()

TypeError: a bytes-like object is required, not '_RSAPublicKey'

#### Questão 2

- Crie um programa que possibilite que o usuário forneça um documento para assinar. Utilize a chave de A para assinar a mensagem e armazene a assinatura em um arquivo binário.

In [16]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

def sing(data):
    # sign the message
    signature = private_key_A.sign(
        data=data.encode('utf-8'),
        padding=padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        algorithm=hashes.SHA256() 
    )
    return signature

# read user document
with open('document.txt', 'r') as file:
    msg = file.read()

# obtain the signature
signature = sing(msg)

# save signature
with open("signature.txt","wb") as file:
    file.write(signature)

#### Questão 3

- Crie um programa que valide a origem do arquivo. Experimente utilizar a chave de A para validar que a mensagem é de autoria de A

In [17]:
from cryptography.exceptions import InvalidSignature

def verify(public_key, data):
    try:
        public_key.verify(
            signature=signature,
            data=msg.encode('utf-8'),
            padding=padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            algorithm=hashes.SHA256()
        )
        print('A mensagem é de autoria de A')
    except InvalidSignature:
        print('A mensagem não é de autoria de A')
        
verify(public_key_A, msg)

A mensagem é de autoria de A


#### Questão 4
- Experimente utilizar a chave B para validar a autoria do arquivo. O que acontece?

In [18]:
verify(public_key_B, msg)

A mensagem não é de autoria de A
